In [ ]:
import pandas as pd
import numpy as np
import datetime
import gzip
import random
import os

dataset = '../datasources/ripulito.csv'


In [ ]:

#PARTIZIONAMENTO IN BASE AL LUOGO
def get_quad_data():
    quads=[ 'Northwest', 'Northeast', 'Southeast']
    struttura = {}
    for quad_value in quads:
        # selezioniamo le righe del DataFrame in base al valore della colonna "QUAD"
        filtered_df = df[df['QUAD'] == quad_value]
        # selezioniamo le colonne richieste
        selected_columns = ['BLOCK', 'WARD', 'DISTRICT', 'PSA', 'ANC', 'NEIGHBORHOOD_CLUSTER', 'BLOCK_GROUP', 'CENSUS_TRACT', 'VOTING_PRECINCT', 'CCN', 'XBLOCK', 'YBLOCK', 'EW', 'NS'] #15
        result_df = filtered_df[selected_columns].dropna()
        list_record = result_df.to_dict('records')
        records=[]
        for record in list_record:
            records.append(list(record.values()))
        struttura[quad_value]=records
    return struttura

#PARTIZIONAMENTO IN BASE AL TIPO
def get_crime_data():
    crimetype=['Non-Violent', 'Violent']
    struttura = {}
    for crime in crimetype:
        filtered_df = df[df['crimetype'] == crime]
        selected_columns=['OFFENSE_TYPE', 'METHOD'] #2
        result_df = filtered_df[selected_columns].dropna()
        list_record = result_df.to_dict('records')
        records=[]
        for record in list_record:
            records.append(list(record.values()))
        struttura[crime]=records
    return struttura

def get_values(struttura, key):
    list=struttura[key]
    index = random.randint(0, len(list)-1)
    return list[index]
    

def generate_record(quad_data, crime_data):
    quad = np.random.choice(['Northwest', 'Northeast', 'Southeast'])
    record1=get_values(quad_data, quad)
    crime=np.random.choice(['Non-Violent', 'Violent'])
    record2=get_values(crime_data, crime)


In [ ]:
def create_crime_csv(timestamp_init, num_rows, chunk_size, shelift_time, quad_data, crime_data):
    num_chunks = num_rows // chunk_size
    date_format = '%Y-%m-%d %H:%M:%S'

    for chunk_num in range(num_chunks):
        print("inizio " + str(chunk_num)+ " chunk di "+str(num_chunks))
        chunk_data = []
        for j in range(chunk_size):
            date = timestamp_init + j*shelift_time+ ( np.random.randint(1, 199))
            date_str = datetime.datetime.fromtimestamp(date).strftime(date_format)
            record = [date_str] + generate_record(quad_data, crime_data)
            chunk_data.append(record)
        chunk_df = pd.DataFrame(chunk_data, columns=['DATE', 'QUAD', 'BLOCK', 'WARD', 'DISTRICT', 'PSA', 'ANC', 'NEIGHBORHOOD_CLUSTER', 'BLOCK_GROUP', 'CENSUS_TRACT', 'VOTING_PRECINCT', 'CCN', 'XBLOCK', 'YBLOCK', 'EW', 'NS', 'CRIMETYPE', 'OFFENSE_TYPE', 'METHOD'])
        chunk_filename = f'../datasources/crime_chunk_{chunk_num}.csv.gz'
        chunk_df.to_csv(chunk_filename, index=False, compression='gzip')
        # Aggiornamento del timestamp_init per il prossimo chunk
        timestamp_init += chunk_size * shelift_time

    # Concatenazione di tutti i chunk in un unico dataframe
    df = pd.concat([pd.read_csv(f'../datasources/crime_chunk_{i}.csv.gz') for i in range(num_chunks)])
    print("fatto")
    # Rimozione dei file temporanei dei chunk
    for i in range(num_chunks):
        os.remove(f'../datasources/crime_chunk_{i}.csv.gz')

    # Esportazione del dataframe in formato CSV compresso
    df.to_csv('../datasources/crimes10.csv.gz', index=False, compression='gzip')

In [ ]:

#anni 90 
#timestamp_init=631152000 
#shelift_time=97
#anni 2020 
#timestamp_init = 1199145600
#shelift_time=470
num_rows=10000000
chunk_size=100000
timestamp_init=631152000 
shelift_time=97
# Creazione del vettore id_vec
id_vec = []

# Lettura del dataset dataset
df = pd.read_csv(dataset)

# Ciclo sui record del dataset
for index, row in df.iterrows():
    # Verifica se ci sono valori nulli nel record
    if row.isnull().values.any():
        print(row['ID'])
        # Inserimento dell'ID nel vettore id_vec
        id_vec.append(row['ID'])
# Rimozione delle righe il cui ID è presente in id_vec
print(len(id_vec))

# Rimozione delle righe il cui ID è presente in id_vec
df = df[~df['ID'].isin(id_vec)]

# Salvataggio del dataframe come file CSV
df.to_csv(dataset, index=False)


quad_data=get_quad_data()
crime_data =get_crime_data()
create_crime_csv(timestamp_init, num_rows, chunk_size, shelift_time, quad_data, crime_data)

In [ ]:
#VEDO INTERVALLO DATE
import pandas as pd

# Carica il dataset in un DataFrame
df = pd.read_csv(dataset)

# Converte la colonna delle date in formato datetime
df['REPORT_DATE'] = pd.to_datetime(df['REPORT_DATE'])

# Trova la data minima e massima
data_minima = df['REPORT_DATE'].min()
data_massima = df['REPORT_DATE'].max()

print("Data minima:", data_minima)
print("Data massima:", data_massima)
